<a href="https://colab.research.google.com/github/Mu1029/Project3/blob/main/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - Object Masking

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# read and display original image

pcb_image_bgr = cv2.imread('drive/MyDrive/Project 3 Data/motherboard_image.JPEG')

pcb_image = cv2.cvtColor(pcb_image_bgr, cv2.COLOR_BGR2RGB)

plt.imshow(pcb_image)
plt.title('Original Image')
plt.axis('off')
plt.show()

In [ ]:
# OpenCV image processing

  # thresholding

pcb_image_gs = cv2.cvtColor(pcb_image, cv2.COLOR_BGR2GRAY)

_, thresh = cv2.threshold(pcb_image_gs, 88, 255, cv2.THRESH_BINARY)

thresh_inv = cv2.bitwise_not(thresh)

  # finding + drawing contours

contours, _ = cv2.findContours(thresh_inv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  # creating + applying mask

mask = np.zeros_like(pcb_image)

cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

result = cv2.bitwise_and(pcb_image, mask)

  # masking background

_, thresh = cv2.threshold(pcb_image_gs, 92, 255, cv2.THRESH_BINARY)

contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

mask = np.zeros_like(pcb_image)

cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

masked_img = cv2.bitwise_and(result, mask)

  # displaying all figures

plt.figure(figsize=(12, 4))

plt.subplot(131)
plt.imshow(thresh, cmap='gray')
plt.title('Thresholded Image')
plt.axis('off')

plt.subplot(132)
plt.imshow(thresh_inv, cmap='gray')
plt.title('Inverted Thresholded Image')
plt.axis('off')

plt.subplot(133)
plt.imshow(masked_img)
plt.title('Object Masking Result')
plt.axis('off')

plt.show()


# Step 2 - YOLOv8 Training

In [2]:
!pip install ultralytics

In [3]:
import ultralytics
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()

ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.0/78.2 GB disk)


In [5]:
# YOLOV8 model training

  # loading pretrained YOLOv8 nano model

model = YOLO('yolov8n.pt')

  # training

model_results = model.train(data='drive/MyDrive/Project 3 Data/data/data.yaml', epochs=100, imgsz=640, batch=5, name='custom_pcb_YOLOv8_model')


Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=drive/MyDrive/Project 3 Data/data/data.yaml, epochs=100, patience=50, batch=5, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_pcb_YOLOv8_model2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

100%|██████████| 755k/755k [00:00<00:00, 18.2MB/s]


Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/drive/MyDrive/Project 3 Data/data/train/labels... 544 images, 0 backgrounds, 0 corrupt: 100%|██████████| 544/544 [02:55<00:00,  3.09it/s]


train: New cache created: /content/drive/MyDrive/Project 3 Data/data/train/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 26, len(boxes) = 108783. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

    100/100      2.57G      1.586      1.104     0.9213        647        640: 100%|██████████| 109/109 [00:50<00:00,  2.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:07<00:00,  1.40it/s]

                   all        105      19108      0.695      0.472      0.549      0.374



100 epochs completed in 1.949 hours.
Optimizer stripped from runs/detect/custom_pcb_YOLOv8_model2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/custom_pcb_YOLOv8_model2/weights/best.pt, 6.3MB

Validating runs/detect/custom_pcb_YOLOv8_model2/weights/best.pt...
Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3008183 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:49<00:00,  4.53s/it]


                   all        105      19108      0.692      0.472      0.547      0.372
                Button        105         45      0.837      0.956      0.976      0.717
             Capacitor        105       7251      0.728      0.217      0.365      0.176
             Connector        105        659      0.697       0.81      0.813      0.582
                 Diode        105         53      0.452      0.113      0.208      0.147
Electrolytic Capacitor        105        160      0.763      0.725      0.813      0.524
                    IC        105       1322      0.703      0.861      0.864      0.612
              Inductor        105         45        0.8      0.178      0.408      0.321
                   Led        105        127      0.746       0.52       0.59      0.367
                  Pads        105        143      0.575     0.0769      0.106     0.0763
                  Pins        105        151      0.527      0.576      0.533      0.322
              Resisto